In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install iterative-stratification

import numpy as np 
import scipy as sp
import sklearn as sl
import pandas as pd
import librosa
from librosa import display
import IPython
import seaborn as sns
import matplotlib.pyplot as plt
from dataclasses import dataclass
from functools import partial
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import tensorflow.data as tfd
import tensorflow_addons as tfa
import tensorflow_hub as hub

import os
from kaggle_datasets import KaggleDatasets

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
VALIDATION = False

cfg = { 
    'Version' : 7,
    'Mode' : 'tpu', #'gpu','cpu'
    'Training' : True,
    'parse_params': {
        'sample_rate' : 48000,
        'cut' : 12.0
    },
    'data_params': {
        'frame_length' : 2048,
        'frame_step' : 512,
        'mel_highest_freq' : 16000.0,#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        'mel_lowest_freq' : 100.0,
        'mel_num_bins' : 256,
        'img_shape' : (256, 512),
        'patch_cut' : 5.5,
        'std' : 0.5,
        'hcrop' : 20,
        'wcrop' : 40,
        'min_seg' : 0.50, 
        'min_fp_seg': 2.0,
        'freqv' : [15000.0, 16000.0, 17000.0, 18000.0,19000.0, 20000.0, 22000.0, 24000.0],    
        'pfp' : 0.2,
        'shuffle_size' : 2048,
    },
    'model_params': {
        'batchsize_per_tpu': 16,
        'iteration_per_epoch': 128,
        'epoch' : 72,
        'ml_model': {
            'fn' : tf.keras.applications.ResNet50,
            'params' : {'include_top': False, 'weights' : 'imagenet', 'pooling' : 'avg' },
            'top_drops' : 0.4,
            'top_size' : 256,
            'name' : 'JustSecondModel'  
        },
        'loss': {
            'fn': tfa.losses.SigmoidFocalCrossEntropy,
            'params': {'reduction' : tf.keras.losses.Reduction.NONE,  'from_logits' : False,},
        },
        'optim': {
            'fn': tfa.optimizers.RectifiedAdam,
            'params': {'lr': 1e-3, 'total_steps': 15*64, 'warmup_proportion': 0.3, 'min_lr': 1e-6},
        },
        'is_tp' : False
    }
}

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path()
AUTOTUNE = tf.data.experimental.AUTOTUNE
SCOPE = tf.name_scope('Not TPU')
NUMBER_OF_REPLICAS = 1
STRATEGY = None
MODE = cfg['Mode']

if MODE == 'tpu':
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    STRATEGY = tf.distribute.experimental.TPUStrategy(tpu)
    SCOPE = STRATEGY.scope()
    NUMBER_OF_REPLICAS = STRATEGY.num_replicas_in_sync
    print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
@dataclass(frozen=True)  # Instances of this class are immutable.
class Params:
    class_n: int = 24
    sr: float = cfg['parse_params']['sample_rate'] # all wave's sample rate may be 48k   
    cut: float = cfg['parse_params']['cut']
    mcut: float = cfg['data_params']['min_seg']
    fpmcut: float = cfg['data_params']['min_fp_seg']
    frm_len: int = cfg['data_params']['frame_length']
    frm_step: int = cfg['data_params']['frame_step']
    mel_frq_h: float = cfg['data_params']['mel_highest_freq']
    mel_frq_l: float = cfg['data_params']['mel_lowest_freq']
    mel_bins: int = cfg['data_params']['img_shape'][0]
    img_size_h: int = cfg['data_params']['img_shape'][0]
    img_size_w: int = cfg['data_params']['img_shape'][1]
    patch_cut: float = cfg['data_params']['patch_cut']
    std: float = cfg['data_params']['std']
    hcrop: int = cfg['data_params']['hcrop']
    wcrop: int = cfg['data_params']['wcrop']
    h: int = cfg['data_params']['img_shape'][0]
    w: int = cfg['data_params']['img_shape'][1]
    frqv = cfg['data_params']['freqv']
    pfp: float = cfg['data_params']['pfp']
    loss_fn = cfg['model_params']['loss']
    optimize_fn = cfg['model_params']['optim']
    training: bool = cfg['Training']
    shuffle: int = cfg['data_params']['shuffle_size']
    version: int = cfg['Version']
    
    @property
    def model_fn(self):
        model_fn = cfg['model_params']['ml_model']
        model_fn['params']['input_shape'] = (self.img_size_h, self.img_size_w, 3)
        return model_fn
        
    @property
    def input_seq_size(self):
        return self.frm_len + (self.img_size_w - 1) * self.frm_step
    @property
    def batch_size(self):
        return cfg['model_params']['batchsize_per_tpu'] * NUMBER_OF_REPLICAS
    @property
    def iter_per_epoch(self):
        return int((cfg['model_params']['iteration_per_epoch'] * self.batch_size) // 128)
    @property
    def batch_val_size(self):
        return NUMBER_OF_REPLICAS * 8

    epoch: int = cfg['model_params']['epoch']
    train_tfrec: str = GCS_DS_PATH + "/tfrecords/train"
    test_tfrec: str = GCS_DS_PATH + "/tfrecords/test"
        
    @property
    def all_train_tfrec(self):
        return [os.path.join(self.train_tfrec,filename) for filename in tf.io.gfile.listdir(self.train_tfrec)]
    
    @property
    def all_test_tfrec(self):
        return [os.path.join(self.test_tfrec,filename) for filename in tf.io.gfile.listdir(self.test_tfrec)]

    path_tp_model: str = ''
    path_fp_model: str = ''
    
    @property
    def path_to_csv_table(self):        
        if cfg['model_params']['is_tp'] == True:
           return '/kaggle/input/rfcx-species-audio-detection/train_tp.csv'
        return '/kaggle/input/rfcx-species-audio-detection/train_fp.csv'
    @property 
    def tp(self):
        if cfg['model_params']['is_tp'] == True:
            return 1
        return 0
    
GPARAMS = Params()

In [ ]:
#[Xmin, Xmax)
class Cropping:
    def __init__(self, params):
        self.in_seq_size = tf.cast(params.cut * params.sr, tf.int32)
        self.min_tp_seq_size = tf.cast(params.mcut * params.sr, tf.int32)
        self.min_fp_seq_size = tf.cast(params.fpmcut * params.sr, tf.int32)
        self.out_seq_size = tf.cast(params.patch_cut * params.sr, tf.int32)
        self.sr = tf.cast(params.sr, tf.float32)
        
    def _find_inertia_position(self, left, right):
        s_diff = tf.cast((self.out_seq_size - (right - left))/2,tf.int32)
        cleft = tf.cast(tf.maximum(0, tf.minimum(left - s_diff,tf.minimum(right + s_diff,self.in_seq_size) - self.out_seq_size)), tf.int32)
        return cleft
    
    def _find_max_left_shift(self, left, right, istp):
        min_seq_size = istp * self.min_tp_seq_size + (1 - istp) * self.min_fp_seq_size
        p = tf.maximum(0, tf.minimum(self.in_seq_size, left + min_seq_size) - self.out_seq_size)
        return p
        
    def _find_max_right_shift(self, left, right, istp):
        min_seq_size = istp * self.min_tp_seq_size + (1 - istp) * self.min_fp_seq_size
        p = tf.maximum(0, tf.minimum(self.in_seq_size, right - min_seq_size + self.out_seq_size) - self.out_seq_size)
        return p
           
    def findRandomPossition(self, t_min, t_max, istp):
        left = tf.cast(t_min * self.sr, tf.int32)
        right = tf.cast(t_max * self.sr, tf.int32)
        p0 = self._find_inertia_position(left, right)
        pleft = self._find_max_left_shift(left, right, istp)
        pright = self._find_max_right_shift(left, right, istp)
        shift = tf.random.uniform([], minval = 0, maxval = 2, dtype = tf.int32)
        p = shift * pleft + (1 - shift) * pright
        pmin = tf.minimum(p0, p)
        pmax = tf.maximum(p0, p)      
        cleft = tf.random.uniform( [], minval = pmin, maxval = pmax + 1, dtype=tf.int32)
        cright = cleft + self.out_seq_size
        return (cleft, cright)
    
    @tf.function
    def __call__(self, sample):
        cleft, cright = self.findRandomPossition(sample['t_min'], sample['t_max'], tf.cast(sample['is_tp'], tf.int32))
        length = cright - cleft
        padding_space = tf.maximum(0, self.out_seq_size - length)
        padding = tf.convert_to_tensor([[0, padding_space]])
        return tf.pad(tf.slice(sample['audio_wav'],[cleft],[length]),padding, "CONSTANT")
#########################################################################################################################################################################
#########################################################################################################################################################################
#########################################################################################################################################################################

class MalSpectrogram:
    def __init__(self, params):
        self.params = params
        self.fft_length = 2 ** int(np.ceil(np.log(params.frm_len) / np.log(2.0)))
        self.num_spectrogram_bins = self.fft_length // 2 + 1
        llmelwm = []
        for frq in params.frqv:
            lmelwm = tf.signal.linear_to_mel_weight_matrix(num_mel_bins = self.params.mel_bins, num_spectrogram_bins = self.num_spectrogram_bins,
                                                           sample_rate = self.params.sr, lower_edge_hertz = self.params.mel_frq_l,upper_edge_hertz = frq)
            llmelwm.append(tf.expand_dims(lmelwm,0))
        self.llmelwm = tf.concat(llmelwm,0)
        
    @tf.function
    def __call__(self, waveform):
        with tf.name_scope('log_mel_features'):
            magnitude_spectrogram = tf.abs(tf.signal.stft(signals = waveform, frame_length = self.params.frm_len,
                                                          frame_step = self.params.frm_step, fft_length=self.fft_length))
            idx = tf.random.uniform([], minval = 0, maxval = len(self.params.frqv), dtype = tf.int32)
            mel_spectrogram = tf.matmul(magnitude_spectrogram, self.llmelwm[idx,...])
            log_mel_spectrogram = tf.math.log(mel_spectrogram + 0.001)
            return tf.transpose(log_mel_spectrogram,[0,2,1])         
#########################################################################################################################################################################
#########################################################################################################################################################################
#########################################################################################################################################################################

class FilterFun:
    def __init__(self, params):
        self.params = params
    
    def __call__(self, recids = None, pfp = 0.0):
        @tf.function
        def _filt_wo_recids(sample):
            flag1 = tf.convert_to_tensor(sample['is_tp'] == 1)
            flag2 = tf.random.uniform([]) < pfp
            return flag1 | flag2      
        @tf.function
        def _filt_with_recids(sample):
            return _filt_wo_recids(sample) & tf.reduce_any(recids == sample['recording_id'])
        if recids is None:
            return _filt_wo_recids
        return _filt_with_recids

In [ ]:
feature_description = {
    'recording_id': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'audio_wav': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'label_info': tf.io.FixedLenFeature([], tf.string, default_value=''),
}

parse_dtype = {
    'audio_wav': tf.float32,
    'recording_id': tf.string,
    'species_id': tf.int32,
    'songtype_id': tf.int32,
    't_min': tf.float32,
    'f_min': tf.float32,
    't_max': tf.float32,
    'f_max':tf.float32,
    'is_tp': tf.int32
}

@tf.function
def parse_step_fun(example_proto, params):
    sample = tf.io.parse_single_example(example_proto, feature_description)
    wav, _ = tf.audio.decode_wav(sample['audio_wav'], desired_channels=1) # mono
    label_info = tf.strings.split(sample['label_info'], sep='"')[1]
    labels = tf.strings.split(label_info, sep=';')
    
    @tf.function
    def _cut_audio(label):
        items = tf.strings.split(label, sep=',')
        spid = tf.squeeze(tf.strings.to_number(items[0], tf.int32))
        soid = tf.squeeze(tf.strings.to_number(items[1], tf.int32))
        tmin = tf.squeeze(tf.strings.to_number(items[2]))
        fmin = tf.squeeze(tf.strings.to_number(items[3]))
        tmax = tf.squeeze(tf.strings.to_number(items[4]))
        fmax = tf.squeeze(tf.strings.to_number(items[5]))
        tp = tf.squeeze(tf.strings.to_number(items[6], tf.int32))

        seq_len = tf.cast(params.cut * params.sr,tf.int32)
        tmax_s = tmax * tf.cast(params.sr, tf.float32)
        tmin_s = tmin * tf.cast(params.sr, tf.float32)
        cut_s = tf.cast(seq_len, tf.float32)
        all_s = tf.cast(60 * params.sr, tf.float32)
        tsize_s = tmax_s - tmin_s
        cut_min = tf.cast(
            tf.maximum(0.0, 
                tf.minimum(tmin_s - (cut_s - tsize_s) / 2,
                           tf.minimum(tmax_s + (cut_s - tsize_s) / 2, all_s) - cut_s)
            ), tf.int32
        )
        
        cut_max = cut_min + seq_len
        
        _sample = {
            'audio_wav': tf.reshape(wav[cut_min:cut_max], [seq_len]),
            'recording_id': sample['recording_id'],
            'species_id': spid,
            'songtype_id': soid,
            't_min': tmin - tf.cast(cut_min, tf.float32)/tf.cast(params.sr, tf.float32),
            'f_min': fmin,
            't_max': tmax - tf.cast(cut_min, tf.float32)/tf.cast(params.sr, tf.float32),
            'f_max': fmax,
            'is_tp': tp
        }
        return _sample
    samples = tf.map_fn(_cut_audio, labels, dtype=parse_dtype)
    return samples


class PipelineComponents:
    def __init__(self, params):
        self.params = params
        self.Crop = Cropping(params)
        self.Spectrogram = MalSpectrogram(params)
        self.FP = FilterFun(params) 
        
    def _striping(self, image):
        h = self.params.img_size_h
        w = self.params.img_size_w 
        hcrop = self.params.hcrop
        wcrop = self.params.wcrop
        offset_x = tf.random.uniform(shape = (), minval = 0, maxval = w - wcrop - 3, dtype = tf.int32)
        width = tf.random.uniform(shape = (), minval = wcrop // 2, maxval = wcrop, dtype = tf.int32 )
        offset_y = tf.random.uniform(shape = (), minval = 0, maxval = h - hcrop - 3, dtype = tf.int32)
        height = tf.random.uniform(shape = (), minval = hcrop // 2, maxval = hcrop, dtype = tf.int32 )
        hor = tf.pad(tensor = tf.zeros([height, w], tf.int32), paddings = [[offset_y, h - offset_y - height],[0,0]], mode='CONSTANT', constant_values=1)
        ver = tf.pad(tensor = tf.zeros([h, width], tf.int32), paddings = [[0,0],[offset_x, w - offset_x - width]], mode='CONSTANT', constant_values=1)
        flag = tf.random.uniform(shape = (), minval = 0, maxval = 2, dtype = tf.int32)
        mask = flag * hor + (1 - flag) * ver
        flag = tf.random.uniform(shape = (), minval = 0, maxval = 2, dtype = tf.int32)
        mask = tf.expand_dims(tf.expand_dims(tf.cast(flag * mask + (1 - flag) * hor * ver, tf.float32),0),-1)
        return image * mask  
        
    def _target_representation(self, sample):       
        return tf.convert_to_tensor([sample['species_id'], sample['is_tp']], dtype = tf.int32)
    
    def CropFn(self):
        @tf.function
        def _crop(sample):
            x = self.Crop(sample)
            y = self._target_representation(sample)
            return { 'x' : x, 'y' : y}
        return _crop
    
    def Output(self):
        @tf.function
        def _output(sample):
            return (sample['x'], sample['y'])
        return _output

    def MapToSpectrogramFn(self):
        out_seq_size = tf.cast(self.params.patch_cut * self.params.sr, tf.int32)
        @tf.function
        def _mapToSpectrogram(sample):
            x = tf.expand_dims(sample['x'], axis = 0)
            x = tf.squeeze(self.Spectrogram(x))
            padding_space =tf.math.maximum(0, self.params.img_size_w - 1 - (out_seq_size - self.params.frm_len)//self.params.frm_step)
            paddings = tf.convert_to_tensor([[0, 0], [0, padding_space]])    
            x = tf.expand_dims(tf.pad(x,paddings)[...,:self.params.img_size_w], 0)
            return {'x' : x, 'y' : sample['y']}
        return _mapToSpectrogram
    
    def AugmentFn(self, training):
        @tf.function
        def _augment(sample):
            img = tf.image.per_image_standardization(tf.expand_dims(sample['x'],-1)) 
            #noskip = tf.cast(1 == tf.cast(training, tf.int32) * sample['y'][-1], tf.bool)
            if training:
                gau = tfkl.GaussianNoise(self.params.std)
                flag = tf.cast(tf.random.uniform(shape = (), minval = 0, maxval = 2, dtype = tf.int32), tf.float32) * tf.cast(sample['y'][-1],tf.float32)
                img = flag * gau(img, training=True) + (1.0 - flag) * img
                flag = tf.cast(tf.random.uniform(shape = (), minval = 0, maxval = 2, dtype = tf.int32), tf.float32) * tf.cast(sample['y'][-1],tf.float32)
                img = flag * self._striping(img) + (1.0 - flag) * img
            img = tf.squeeze(img)
            img = (img - tf.math.reduce_min(img)) / (tf.math.reduce_max(img) - tf.math.reduce_min(img))
            return (tf.reshape(img,[self.params.h, self.params.w]), sample["y"])
        return _augment

In [ ]:
class DatasetSrc:
    def __init__(self, params):
        self.Params = params
        self.Filters = PipelineComponents(params)
        self.BaseDataSet = (tfd.TFRecordDataset(params.all_train_tfrec)
                            .map(partial(parse_step_fun, params = self.Params), num_parallel_calls=AUTOTUNE)
                            .unbatch())
        
    def CreateDataset(self):
        tr_dataset = (self.BaseDataSet.filter(self.Filters.FP(None, self.Params.pfp))
                      .cache()
                      .map(self.Filters.CropFn(),num_parallel_calls=AUTOTUNE)
                      .map(self.Filters.MapToSpectrogramFn(),num_parallel_calls=AUTOTUNE)
                      #.map(self.Filters.Output(),num_parallel_calls=AUTOTUNE)
                      .map(self.Filters.AugmentFn(True),num_parallel_calls=AUTOTUNE)
                      .batch(self.Params.batch_size).prefetch(AUTOTUNE))
        return tr_dataset
    
    def CreateTrainDataset(self, shufflelen = None, recordIdv = None):
        tr_dataset = (self.BaseDataSet.filter(self.Filters.FP(recordIdv, self.Params.pfp))
                      .cache()
                      .map(self.Filters.CropFn()))
        if shufflelen is not None:
            tr_dataset = tr_dataset.shuffle(shufflelen)
        tr_dataset = (tr_dataset.repeat()
                      .map(self.Filters.MapToSpectrogramFn(),num_parallel_calls=AUTOTUNE)
                      .map(self.Filters.AugmentFn(True),num_parallel_calls=AUTOTUNE) 
                      .batch(self.Params.batch_size).prefetch(AUTOTUNE))
        return tr_dataset
 
    def CreateValDataset(self, recordIdv = None):
        val_dataset = (self.BaseDataSet.filter(self.FP(recordIdv))
                      .map(self.Filters.CropFn())
                      .map(self.Filters.MapToSpectrogramFn(),num_parallel_calls=AUTOTUNE)
                      .map(self.Filters.AugmentFn(False),num_parallel_calls=AUTOTUNE)
                      .batch(self.Params.batch_val_size).cache())
        return val_dataset

# Metric

In [ ]:
@tf.function
def _one_sample_positive_class_precisions(example):
    y_true, y_pred = example
    y_true = tf.cast(y_true > 0, tf.float32)

    retrieved_classes = tf.argsort(y_pred, direction='DESCENDING')
    class_rankings = tf.argsort(retrieved_classes)
    retrieved_class_true = tf.gather(y_true, retrieved_classes)
    retrieved_cumulative_hits = tf.math.cumsum(tf.cast(retrieved_class_true, tf.float32))

    idx = tf.where(y_true)[:, 0]
    i = tf.boolean_mask(class_rankings, y_true)
    r = tf.gather(retrieved_cumulative_hits, i)
    c = 1 + tf.cast(i, tf.float32)
    precisions = r / c

    dense = tf.scatter_nd(idx[:, None], precisions, [y_pred.shape[0]])
    return dense

class LWLRAP(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='lwlrap'):
        super().__init__(name=name)

        self._precisions = self.add_weight(
            name='per_class_cumulative_precision',
            shape=[num_classes],
            initializer='zeros',
        )

        self._counts = self.add_weight(
            name='per_class_cumulative_count',
            shape=[num_classes],
            initializer='zeros',
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        precisions = tf.map_fn(
            fn=_one_sample_positive_class_precisions,
            elems=(y_true, y_pred),
            dtype=(tf.float32),
        )

        increments = tf.cast(precisions > 0, tf.float32)
        total_increments = tf.reduce_sum(increments, axis=0)
        total_precisions = tf.reduce_sum(precisions, axis=0)

        self._precisions.assign_add(total_precisions)
        self._counts.assign_add(total_increments)        

    def result(self):
        per_class_lwlrap = self._precisions / tf.maximum(self._counts, 1.0)
        per_class_weight = self._counts / tf.reduce_sum(self._counts)
        overall_lwlrap = tf.reduce_sum(per_class_lwlrap * per_class_weight)
        return overall_lwlrap

    def reset_states(self):
        self._precisions.assign(self._precisions * 0)
        self._counts.assign(self._counts * 0)

# Utils

In [ ]:
def plot_history(history, name):
    plt.figure(figsize=(8,3))
    plt.subplot(1,2,1)
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.title("loss")
    # plt.yscale('log')

    plt.subplot(1,2,2)
    plt.plot(history.history["lwlrap"])
    plt.plot(history.history["val_lwlrap"])
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.title("metric")

    plt.savefig(name)
    
def plot_train_history(history, name):
    plt.figure(figsize=(8,3))
    plt.subplot(1,2,1)
    plt.plot(history.history["loss"])
    plt.title("loss")
    # plt.yscale('log')

    plt.subplot(1,2,2)
    plt.plot(history.history["lwlrap"])
    plt.title("metric")

    plt.savefig(name)
    
def template_save_weights(curepoch, epochs, model_step, pfp = '05',min_seg = '05', min_seg_fp = '2', units = 256,
                          shuffle = GPARAMS.shuffle, version = GPARAMS.version, tp = GPARAMS.tp):
    if curepoch in epochs:
        filename = f'model_v{version}_e{curepoch}_s{shuffle}_ms{min_seg}_msfp{min_seg_fp}_u{units}_tp{tp}_pfp{pfp}.h5'
        model_step.model.save_weights(filename)
        print(filename)

# Machine Learning Model

In [ ]:
class ComponentFactory:
    def __init__(self, params):
        self.params = params
        self.loss = None
        self.optimize = None
        self.model = None
        self.train_acc = None
        self.test_acc = None
        
    def _create_model(self):
        params = self.params
        cnn_model = params.model_fn['fn'](**params.model_fn['params'])
        for layer in cnn_model.layers:
            layer.trainable = True
        inputs = tfkl.Input(shape = (params.img_size_h, params.img_size_w))
        x = tfkl.Reshape((params.img_size_h, params.img_size_w,1))(inputs)
        x = tfkl.Conv2D(filters = 3, kernel_size = 7, padding='same', activation = 'sigmoid')(x)
        x = cnn_model(x)
        outputs = []
        for _ in range(params.class_n):
            t = tfkl.Dropout(0.4)(x, training = params.training)
            t = tfkl.Dense(params.model_fn['top_size'], activation='relu')(t)
            t = tfkl.Dropout(params.model_fn['top_drops'])(t, training = params.training)
            t = tfkl.Dense(1, activation = 'sigmoid')(t)
            outputs.append(t)
        model = tfk.Model(name=params.model_fn['name'], inputs=inputs,outputs=outputs)
        return model
    
    def CreateModelIngradients(self):
        self.model = self._create_model()
        loss_obj = self.params.loss_fn['fn'](**self.params.loss_fn['params'])
        def _loss_fn(y_true, y_pred):
            return tf.nn.compute_average_loss(loss_obj(y_true = y_true,y_pred = y_pred),global_batch_size = self.params.batch_size)
        self.loss = _loss_fn
        optimizer = self.params.optimize_fn['fn'](**self.params.optimize_fn['params'])
        def _opt_fn(gradients):
            return optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        self.optimize = _opt_fn
        self.train_acc = LWLRAP(self.params.class_n, name = 'train_lwlrap')
        self.test_acc = LWLRAP(self.params.class_n, name = 'test_lwlrap')
        return self
        
    def CreateModelIngradientsInScope(self):
        with SCOPE:
            self.CreateModelIngradients()
        return self 
    
class CreateStepFns:
    def __init__(self, ingradients):
        self.model = ingradients.model
        self.loss = ingradients.loss
        self.optimize = ingradients.optimize
        self.train_acc = ingradients.train_acc
        self.test_acc = ingradients.test_acc
        self.class_n = ingradients.params.class_n
        self.tp = ingradients.params.tp == 1
        
    def _train_step_tp(self, inputs):
        images, y = inputs
        y_true = tf.one_hot(y[:,0],self.class_n, on_value = 1.0, off_value = 0.0)
        with tf.GradientTape() as tape:
            y_pred = self.model(images, training=True)
            y_pred = tf.concat(y_pred, axis = -1)
            loss = self.loss(y_true, y_pred)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimize(gradients)
        self.train_acc.update_state(y_true, y_pred)
        return loss
    
    def _train_step_tp_fp(self,inputs):
        images, y = inputs
        y_true = tf.one_hot(y[:,0] + self.class_n * (y[:,1] - 1),self.class_n, on_value = 1.0, off_value = 0.0)
        ix1 = tf.cast(tf.where(y[:,1] == 1), tf.int32)
        y_true1 = tf.squeeze(tf.one_hot(tf.gather_nd(y[:,0], tf.expand_dims(ix1, -1)), self.class_n, on_value = 1.0, off_value = 0.0))

        ix2 = tf.cast(tf.where(y[:,1] == 0),tf.int32)
        ys = tf.gather_nd(y[:,0], tf.expand_dims(ix2, -1))
        y_true2 = tf.zeros_like(ys,dtype = tf.float32)
        idxx = tf.stack([ix2, ys], axis = -1)
        with tf.GradientTape() as tape:
            y_pred = tf.concat(self.model(images,training = True),axis = -1)
            y_pred1 = tf.squeeze(tf.gather_nd(y_pred, tf.expand_dims(ix1, -1)))
            y_pred2 = tf.gather_nd(y_pred, idxx)
            loss1 = self.loss(y_true1, y_pred1)
            loss2 = self.loss(y_true2, y_pred2)
            loss = loss1 + loss2
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimize(gradients)
        self.train_acc.update_state(y_true, y_pred)
        return loss
           
    def CreateTrainStepFn(self):
        if self.tp == True:
            return lambda inputs : self._train_step_tp(inputs)
        if self.tp == False:
            return lambda inputs : self._train_step_tp_fp(inputs)
        
    def CreateTestStepFn(self):
        def _test_step(inputs):
            images, y_true = inputs
            y_pred = self.model(images, training = False)
            y_pred = tf.concat(y_pred, axis = -1)
            self.test_acc.update_state(y_true, y_pred)
        return _test_step   
#&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&    
#ml_ingradients = ComponentFactory(GPARAMS).CreateModelIngradientsInScope()

# Training Data preparation

In [ ]:
dataset = DatasetSrc(GPARAMS)
df = pd.read_csv(GPARAMS.path_to_csv_table)
table = df.groupby('recording_id')['species_id'].apply(lambda x: tf.scatter_nd(tf.expand_dims(x, 1),
                                                                               np.ones_like(x),
                                                                               shape=[GPARAMS.class_n]).numpy()).reset_index()

skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=0)
idx_splits = list(skf.split(table.recording_id, np.stack(table.species_id.to_numpy())))
splits = list(map(lambda xs: (table.recording_id[xs[0]].to_numpy(), table.recording_id[xs[1]].to_numpy()), idx_splits))

In [ ]:
train_dataset = None
test_dataset = None
tr_datasets = []
val_datasets = []

if VALIDATION:
    for i in range(5):
        tr_dataset = dataset.CreateTrainDataset(splits[i][0], shufflelen = GPARAMS.shuffle // 2)
        tr_datasets.append(tr_dataset)
    
    for i in range(5):
        val_dataset = dataset.CreateValDataset(splits[i][1])
        val_datasets.append(val_dataset)

In [ ]:
idx = 1

if VALIDATION:
    train_dataset = tr_datasets[idx]
    test_dataset = val_datasets[idx]
else:
    train_dataset = dataset.CreateTrainDataset(shufflelen = GPARAMS.shuffle)
    
if MODE == 'tpu':
    train_dataset = STRATEGY.experimental_distribute_dataset(train_dataset)
    if VALIDATION:
        test_dataset = STRATEGY.experimental_distribute_dataset(test_dataset)

# Training ML Model Creation and Initialization

In [ ]:
ml_ingradients = ComponentFactory(GPARAMS).CreateModelIngradientsInScope()
ml_steps = CreateStepFns(ml_ingradients)
train_step = ml_steps.CreateTrainStepFn()
test_step = ml_steps.CreateTestStepFn()

@tf.function
def distributed_train_step(dataset_inputs):
    losses = train_step(dataset_inputs)
    return losses

@tf.function
def distributed_test_step(dataset_inputs):
    test_step(dataset_inputs)

if MODE == 'tpu':
    @tf.function
    def distributed_train_step(dataset_inputs):
      per_replica_losses = STRATEGY.run(train_step, args=(dataset_inputs,))
      return STRATEGY.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,axis=None)

    @tf.function
    def distributed_test_step(dataset_inputs):
        return STRATEGY.run(test_step, args=(dataset_inputs,))

In [ ]:
epochs = [24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72]
save_weights = partial(template_save_weights,epochs = epochs, model_step = ml_steps, min_seg = '05', pfp = '02', min_seg_fp = '2', units = 256)

train_acc = ml_ingradients.train_acc
test_acc = ml_ingradients.test_acc

for epoch in range(GPARAMS.epoch):
  # TRAIN LOOP
    total_loss = 0.0
    num_batches = 0
    train_it = iter(train_dataset)
    for _ in range(GPARAMS.iter_per_epoch):
        total_loss += distributed_train_step(next(train_it))
        num_batches += 1
    train_loss = total_loss / num_batches 

    if VALIDATION:
        for x in test_dataset:
            distributed_test_step(x)

    template = "Epoch {}, Loss: {}, Train_Acc: {}, Test_Acc: {}"
    print (template.format(epoch+1, train_loss,train_acc.result()*100,test_acc.result()*100))

    train_acc.reset_states()
    test_acc.reset_states()
    
    save_weights(epoch)

In [ ]:
ml_steps.model.save_weights('modelv7_e72_i128_fp_s512_s2048_minseg05_u256_fpf02.h5')

In [ ]:
'''
dataset = DatasetSrc(GPARAMS).CreateTrainDataset()
b1 = next(iter(dataset))
img, y = b1

ix1 = tf.cast(tf.where(y[:,1] == 1), tf.int32)
#ix1 = tf.cast(tf.where([False] * 16),tf.int32)
y_true1 = tf.squeeze(tf.one_hot(tf.gather_nd(y[:,0], tf.expand_dims(ix1, -1)),24, on_value = 1.0, off_value = 0.0))

ix2 = tf.cast(tf.where(y[:,1] == 0),tf.int32)
#ix2 = tf.cast(tf.where([True] * 16),tf.int32)
ys = tf.gather_nd(y[:,0], tf.expand_dims(ix2, -1))
y_true2 = tf.zeros_like(ys,dtype = tf.float32)
idxx = tf.stack([ix2, ys], axis = -1)
with tf.GradientTape() as tape:
    pred = ml_steps.model(img,training = True)
    pred1 = tf.concat(pred, axis = -1)

    y_pred1 = tf.squeeze(tf.gather_nd(pred1, tf.expand_dims(ix1, -1)))
    y_pred2 = tf.gather_nd(pred1, idxx)

    loss1 = ml_steps.loss(y_true1, y_pred1)
    loss2 = ml_steps.loss(y_true2, y_pred2)
    loss = loss1 + loss2
gradients = tape.gradient(loss2, ml_steps.model.trainable_variables)
print(tf.shape(y_true1))
print(tf.shape(y_pred1))
print(loss)

res = 0.0
for gr in gradients:
    res = res + tf.math.reduce_sum(tf.math.abs(gr))
print(res)
'''

In [ ]:
'''
    def CreateTrainStepFn(self):
        def _train_step(inputs):
            images, y = inputs
            y_true = tf.one_hot(y[:,0],self.class_n, on_value = 1.0, off_value = 0.0)
            with tf.GradientTape() as tape:
                y_pred = self.model(images, training=True)
                y_pred = tf.concat(y_pred, axis = -1)
                loss = self.loss(y_true, y_pred)

            gradients = tape.gradient(loss, self.model.trainable_variables)
            self.optimize(gradients)
            self.train_acc.update_state(y_true, y_pred)
            return loss
        return _train_step
'''

In [ ]:
tr_dataset =  DatasetSrc(GPARAMS).CreateUnbatchDataset().batch(4)
sample = next(iter(tr_dataset))

In [ ]:
def compute_loss(labels, predictions):
    loss = cfg['model_params']['loss']['fn'](**cfg['model_params']['loss']['params'])
    return tf.nn.compute_average_loss(loss(y_true = labels,y_pred = predictions))

optimizer = cfg['model_params']['optim']['fn'](**cfg['model_params']['optim']['params'])

def train_step(inputs, model, optimizer = optimizer):
    images, targets = inputs
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = compute_loss(targets, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss

In [ ]:
idx = tf.math.argmax(sample[1],-1)
idx = tf.convert_to_tensor([[0,14]])#,[1,23]])
predictions = tf.reshape(tf.transpose(model(sample[0], training=True),[1,2,0]), [2,24])
pred1 = tf.gather_nd(predictions, idx)
pred2 = predictions[1,...]
loss1 = compute_loss([[1.0]], pred1)
loss2 = compute_loss(tf.zeros([1,24]), pred2)
loss = loss1 + loss2

In [ ]:
with SCOPE:
    cnn_model = cfg['model_params']['ml_model'](include_top = False, weights = 'imagenet', pooling='avg',
                                           input_shape = (GPARAMS.img_size_h, GPARAMS.img_size_h, 3))
    model = create_model(cnn_model,GPARAMS)
    optimizer = cfg['model_params']['optim']['fn'](**cfg['model_params']['optim']['params'])
    loss = cfg['model_params']['loss']['fn'](**cfg['model_params']['loss']['params'])
    model.compile(optimizer=optimizer,loss=loss, metrics=[LWLRAP(GPARAMS.class_n)])

In [ ]:
history = model.fit(tr_dataset,steps_per_epoch = GPARAMS.iter_per_epoch,epochs = GPARAMS.epoch)
model.save_weights('model_aug_e16_i64_tp_s512_f18000_v2.h5')

In [ ]:
plot_train_history(history, 'Just_train_e16_i64_16000_augcor')

In [ ]:
'''
def train_step(inputs, model, optimizer = optimizer):
    images = inputs[0]
    idx = tf.math.argmax(inputs[1],-1)
    idx1 = tf.convert_to_tensor([[0,21]])
    idx2 = tf.convert_to_tensor([[1,23]])
    with tf.GradientTape() as tape:
        predictions = tf.reshape(tf.transpose(model(sample[0], training=True),[1,2,0]), [2,24])
        pred1 = tf.gather_nd(predictions, idx1)
        pred2 = tf.gather_nd(predictions, idx2)#predictions[1,...]
        loss1 = compute_loss([[1.0]], pred1)
        loss2 = compute_loss([[1.0]], pred2)#compute_loss(tf.zeros([1,24]), pred2)
        loss = loss1 + loss2

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return (loss, gradients)
'''